# Neural Machine Translation

In [3]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

### Dataset

In [4]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 28404.51it/s]


In [6]:
dataset[:5]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26')]

In [12]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("Source date:", dataset[0][0])
print("Target date:", dataset[0][1])
print()
print("Source after preprocessing (indices):", X[0])
print("Target after preprocessing (indices):", Y[0])
print()
print("Source after preprocessing (one-hot):", Xoh[0])
print("Target after preprocessing (one-hot):", Yoh[0])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Attention mechanism

In [14]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [15]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e.
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas"
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a", in this order, to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas, a])
    
    return context

### Model

In [16]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [26]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # Define your pre-attention Bi-LSTM.
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    # Iterate for Ty steps
    for t in range(Ty):
    
        # Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # Apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        
        # Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(inputs=s)
        
        # Append "out" to the "outputs" list
        outputs.append(out)
    
    # Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0 ,c0], outputs=outputs)
    
    return model

In [27]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30, 37)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 30, 64)      17920       ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 repeat_vector (RepeatVector)   (None, 30, 64)       0           ['s0[0][0]',               

                                                                  'attention_weights[14][0]',     
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[15][0]',     
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[16][0]',     
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[17][0]',     
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[18][0]',     
                                                                  'bidirectional_1[0][0]',        
          

In [28]:
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [29]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [30]:
model.fit([Xoh, s0, c0], outputs, epochs=100, batch_size=100)

Epoch 1/100
100/100 [==============================] - 15s 29ms/step - loss: 6.9723 - dense_3_loss: 0.3010 - dense_3_1_loss: 0.2338 - dense_3_2_loss: 0.7544 - dense_3_3_loss: 1.0779 - dense_3_4_loss: 0.0427 - dense_3_5_loss: 0.2647 - dense_3_6_loss: 1.4608 - dense_3_7_loss: 0.0653 - dense_3_8_loss: 1.0647 - dense_3_9_loss: 1.7070 - dense_3_accuracy: 0.9242 - dense_3_1_accuracy: 0.9262 - dense_3_2_accuracy: 0.7357 - dense_3_3_accuracy: 0.6590 - dense_3_4_accuracy: 0.9983 - dense_3_5_accuracy: 0.8967 - dense_3_6_accuracy: 0.4954 - dense_3_7_accuracy: 0.9906 - dense_3_8_accuracy: 0.5448 - dense_3_9_accuracy: 0.4088
Epoch 2/100
100/100 [==============================] - 3s 28ms/step - loss: 0.8171 - dense_3_loss: 0.0268 - dense_3_1_loss: 0.0082 - dense_3_2_loss: 0.0481 - dense_3_3_loss: 0.0777 - dense_3_4_loss: 9.4094e-05 - dense_3_5_loss: 0.0566 - dense_3_6_loss: 0.2398 - dense_3_7_loss: 1.4058e-04 - dense_3_8_loss: 0.2040 - dense_3_9_loss: 0.1557 - dense_3_accuracy: 0.9895 - dense_3_1_ac

100/100 [==============================] - 3s 29ms/step - loss: 0.0394 - dense_3_loss: 0.0016 - dense_3_1_loss: 2.1765e-04 - dense_3_2_loss: 0.0035 - dense_3_3_loss: 0.0196 - dense_3_4_loss: 3.5981e-05 - dense_3_5_loss: 0.0055 - dense_3_6_loss: 0.0041 - dense_3_7_loss: 1.3346e-04 - dense_3_8_loss: 0.0025 - dense_3_9_loss: 0.0022 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 0.9983 - dense_3_6_accuracy: 0.9998 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 0.9999 - dense_3_9_accuracy: 1.0000
Epoch 15/100
100/100 [==============================] - 3s 28ms/step - loss: 0.0383 - dense_3_loss: 0.0014 - dense_3_1_loss: 2.0740e-04 - dense_3_2_loss: 0.0034 - dense_3_3_loss: 0.0193 - dense_3_4_loss: 3.2207e-05 - dense_3_5_loss: 0.0057 - dense_3_6_loss: 0.0038 - dense_3_7_loss: 1.2613e-04 - dense_3_8_loss: 0.0023 - dense_3_9_loss: 0.0020 - dense_3_accuracy: 1.0000 - dense_3_

100/100 [==============================] - 3s 28ms/step - loss: 0.0279 - dense_3_loss: 7.7540e-04 - dense_3_1_loss: 1.2873e-04 - dense_3_2_loss: 0.0029 - dense_3_3_loss: 0.0168 - dense_3_4_loss: 2.7682e-05 - dense_3_5_loss: 0.0029 - dense_3_6_loss: 0.0020 - dense_3_7_loss: 8.4687e-05 - dense_3_8_loss: 0.0012 - dense_3_9_loss: 0.0011 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 0.9994 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 27/100
100/100 [==============================] - 3s 28ms/step - loss: 0.0275 - dense_3_loss: 7.5215e-04 - dense_3_1_loss: 1.3588e-04 - dense_3_2_loss: 0.0029 - dense_3_3_loss: 0.0164 - dense_3_4_loss: 2.8901e-05 - dense_3_5_loss: 0.0029 - dense_3_6_loss: 0.0020 - dense_3_7_loss: 8.9110e-05 - dense_3_8_loss: 0.0012 - dense_3_9_loss: 0.0010 - dense_3_accuracy: 1.0000 - 

100/100 [==============================] - 3s 29ms/step - loss: 0.0224 - dense_3_loss: 4.9580e-04 - dense_3_1_loss: 1.1677e-04 - dense_3_2_loss: 0.0025 - dense_3_3_loss: 0.0148 - dense_3_4_loss: 2.4543e-05 - dense_3_5_loss: 0.0015 - dense_3_6_loss: 0.0014 - dense_3_7_loss: 6.6860e-05 - dense_3_8_loss: 7.1447e-04 - dense_3_9_loss: 7.3113e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 0.9999 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 39/100
100/100 [==============================] - 3s 30ms/step - loss: 0.0224 - dense_3_loss: 4.8052e-04 - dense_3_1_loss: 1.0851e-04 - dense_3_2_loss: 0.0025 - dense_3_3_loss: 0.0147 - dense_3_4_loss: 2.5413e-05 - dense_3_5_loss: 0.0019 - dense_3_6_loss: 0.0013 - dense_3_7_loss: 6.4379e-05 - dense_3_8_loss: 6.8422e-04 - dense_3_9_loss: 6.9224e-04 - dense_3_acc

100/100 [==============================] - 3s 30ms/step - loss: 0.0191 - dense_3_loss: 3.5701e-04 - dense_3_1_loss: 9.5976e-05 - dense_3_2_loss: 0.0023 - dense_3_3_loss: 0.0131 - dense_3_4_loss: 2.1343e-05 - dense_3_5_loss: 0.0011 - dense_3_6_loss: 9.6223e-04 - dense_3_7_loss: 5.0211e-05 - dense_3_8_loss: 5.1103e-04 - dense_3_9_loss: 5.4117e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 1.0000 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 51/100
100/100 [==============================] - 3s 29ms/step - loss: 0.0185 - dense_3_loss: 3.4271e-04 - dense_3_1_loss: 9.6587e-05 - dense_3_2_loss: 0.0022 - dense_3_3_loss: 0.0129 - dense_3_4_loss: 1.9052e-05 - dense_3_5_loss: 0.0010 - dense_3_6_loss: 9.0262e-04 - dense_3_7_loss: 4.6736e-05 - dense_3_8_loss: 4.9556e-04 - dense_3_9_loss: 5.2373e-04 - den

100/100 [==============================] - 3s 31ms/step - loss: 0.0158 - dense_3_loss: 2.6907e-04 - dense_3_1_loss: 9.2603e-05 - dense_3_2_loss: 0.0019 - dense_3_3_loss: 0.0114 - dense_3_4_loss: 1.7177e-05 - dense_3_5_loss: 6.0794e-04 - dense_3_6_loss: 6.9808e-04 - dense_3_7_loss: 4.0961e-05 - dense_3_8_loss: 3.6526e-04 - dense_3_9_loss: 4.1930e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 1.0000 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 63/100
100/100 [==============================] - 3s 34ms/step - loss: 0.0154 - dense_3_loss: 2.6448e-04 - dense_3_1_loss: 9.0830e-05 - dense_3_2_loss: 0.0018 - dense_3_3_loss: 0.0111 - dense_3_4_loss: 1.7187e-05 - dense_3_5_loss: 6.4052e-04 - dense_3_6_loss: 6.9419e-04 - dense_3_7_loss: 3.7488e-05 - dense_3_8_loss: 3.5687e-04 - dense_3_9_loss: 4.0678e-

100/100 [==============================] - 3s 30ms/step - loss: 0.0132 - dense_3_loss: 2.2667e-04 - dense_3_1_loss: 7.9169e-05 - dense_3_2_loss: 0.0017 - dense_3_3_loss: 0.0095 - dense_3_4_loss: 1.5134e-05 - dense_3_5_loss: 4.9110e-04 - dense_3_6_loss: 5.5800e-04 - dense_3_7_loss: 2.6129e-05 - dense_3_8_loss: 2.8037e-04 - dense_3_9_loss: 3.4543e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9976 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 1.0000 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 75/100
100/100 [==============================] - 3s 30ms/step - loss: 0.0129 - dense_3_loss: 2.1978e-04 - dense_3_1_loss: 8.4014e-05 - dense_3_2_loss: 0.0016 - dense_3_3_loss: 0.0094 - dense_3_4_loss: 1.5759e-05 - dense_3_5_loss: 4.5239e-04 - dense_3_6_loss: 5.4541e-04 - dense_3_7_loss: 2.8032e-05 - dense_3_8_loss: 2.7000e-04 - dense_3_9_loss: 3.3983e-

100/100 [==============================] - 3s 30ms/step - loss: 0.0109 - dense_3_loss: 1.9053e-04 - dense_3_1_loss: 7.6455e-05 - dense_3_2_loss: 0.0014 - dense_3_3_loss: 0.0079 - dense_3_4_loss: 1.2059e-05 - dense_3_5_loss: 3.4778e-04 - dense_3_6_loss: 4.5883e-04 - dense_3_7_loss: 2.1561e-05 - dense_3_8_loss: 2.1028e-04 - dense_3_9_loss: 2.8667e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9977 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 1.0000 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 87/100
100/100 [==============================] - 3s 30ms/step - loss: 0.0109 - dense_3_loss: 1.8970e-04 - dense_3_1_loss: 7.7551e-05 - dense_3_2_loss: 0.0014 - dense_3_3_loss: 0.0079 - dense_3_4_loss: 1.0426e-05 - dense_3_5_loss: 3.3904e-04 - dense_3_6_loss: 4.3784e-04 - dense_3_7_loss: 2.2740e-05 - dense_3_8_loss: 2.1243e-04 - dense_3_9_loss: 2.8166e-

100/100 [==============================] - 3s 31ms/step - loss: 0.0094 - dense_3_loss: 1.6646e-04 - dense_3_1_loss: 7.1790e-05 - dense_3_2_loss: 0.0013 - dense_3_3_loss: 0.0068 - dense_3_4_loss: 9.3703e-06 - dense_3_5_loss: 2.9044e-04 - dense_3_6_loss: 3.7227e-04 - dense_3_7_loss: 1.6030e-05 - dense_3_8_loss: 1.7119e-04 - dense_3_9_loss: 2.4840e-04 - dense_3_accuracy: 1.0000 - dense_3_1_accuracy: 1.0000 - dense_3_2_accuracy: 0.9997 - dense_3_3_accuracy: 0.9978 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 1.0000 - dense_3_6_accuracy: 1.0000 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 1.0000 - dense_3_9_accuracy: 1.0000
Epoch 99/100
100/100 [==============================] - 3s 32ms/step - loss: 0.0094 - dense_3_loss: 1.6632e-04 - dense_3_1_loss: 7.4128e-05 - dense_3_2_loss: 0.0012 - dense_3_3_loss: 0.0068 - dense_3_4_loss: 9.9855e-06 - dense_3_5_loss: 2.9125e-04 - dense_3_6_loss: 3.9370e-04 - dense_3_7_loss: 1.5880e-05 - dense_3_8_loss: 1.9647e-04 - dense_3_9_loss: 2.4973e-

In [31]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

source: 3 May 1979
output: 1979-05-03 

source: 5 April 09
output: 2009-05-05 

source: 21th of August 2016
output: 2016-08-02 

source: Tue 10 Jul 2007
output: 2007-07-10 

source: Saturday May 9 2018
output: 2018-05-09 

source: March 3 2001
output: 2001-03-03 

source: March 3rd 2001
output: 2001-03-03 

source: 1 March 2001
output: 2001-03-01 

